In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Generate synthetic dataset
X, y = make_classification(n_samples=500, n_features=2, n_classes=2, n_informative=2, n_redundant=0, random_state=42)
y = 2 * y - 1  # Convert labels from (0,1) to (-1,1)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

# Define the SVM as a simple linear model
class SVM_NN(nn.Module):
    def __init__(self, input_dim):
        super(SVM_NN, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Linear transformation

    def forward(self, x):
        return self.linear(x)  # No activation function

# Hinge loss function
def hinge_loss(output, target):
    return torch.mean(torch.clamp(1 - target * output, min=0))  # max(0, 1 - y * (w^T x + b))

# Initialize model, optimizer
model = SVM_NN(input_dim=2)
optimizer = optim.Adam(model.parameters(), lr=0.01)
lambda_reg = 0.01  # Regularization term

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()

    output = model(X_train_tensor)
    loss = hinge_loss(output, y_train_tensor) + lambda_reg * torch.norm(model.linear.weight, p=2)  # Hinge loss + L2 regularization

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate
with torch.no_grad():
    y_pred = model(X_test_tensor).sign().numpy()
    accuracy = np.mean(y_pred == y_test.reshape(-1, 1))
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


Epoch [10/100], Loss: 0.5690
Epoch [20/100], Loss: 0.4954
Epoch [30/100], Loss: 0.4410
Epoch [40/100], Loss: 0.4087
Epoch [50/100], Loss: 0.3847
Epoch [60/100], Loss: 0.3681
Epoch [70/100], Loss: 0.3570
Epoch [80/100], Loss: 0.3482
Epoch [90/100], Loss: 0.3414
Epoch [100/100], Loss: 0.3358
Test Accuracy: 88.00%


In [2]:
from sklearn.metrics import mean_squared_error

# Generate synthetic data (same as before)
np.random.seed(42)
X = np.sort(5 * np.random.rand(100, 1), axis=0)  # Features (1D)
y = np.sin(X).ravel() + 0.1 * np.random.randn(100)  # Target with noise

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.reshape(-1, 1)).ravel()

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_scaled, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)  # Keep in original scale for evaluation

# Define SVR-like Neural Network
class SVR_NN(nn.Module):
    def __init__(self, input_dim):
        super(SVR_NN, self).__init__()
        self.linear = nn.Linear(input_dim, 1)  # Simple linear regression layer

    def forward(self, x):
        return self.linear(x)

# SVR Loss (ε-insensitive loss)
class SVRLoss(nn.Module):
    def __init__(self, epsilon=0.1, lambda_reg=0.01):
        super(SVRLoss, self).__init__()
        self.epsilon = epsilon
        self.lambda_reg = lambda_reg

    def forward(self, predictions, targets, model):
        abs_error = torch.abs(predictions - targets)
        loss = torch.mean(torch.clamp(abs_error - self.epsilon, min=0))  # Max(0, |y - f(X)| - epsilon)
        reg = self.lambda_reg * torch.norm(model.linear.weight, p=2)  # L2 regularization
        return loss + reg

# Initialize model, loss, and optimizer
model = SVR_NN(input_dim=1)
criterion = SVRLoss(epsilon=0.1, lambda_reg=0.01)
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()

    predictions = model(X_train_tensor)
    loss = criterion(predictions, y_train_tensor, model)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

# Evaluate on test data
with torch.no_grad():
    y_pred_scaled = model(X_test_tensor).numpy()
    y_pred = scaler_y.inverse_transform(y_pred_scaled)  # Convert back to original scale
    mse = mean_squared_error(y_test,y_pred)
    print(f"mse: {mse:.4f}")

Epoch [10/100], Loss: 0.7175
Epoch [20/100], Loss: 0.6451
Epoch [30/100], Loss: 0.5921
Epoch [40/100], Loss: 0.5532
Epoch [50/100], Loss: 0.5236
Epoch [60/100], Loss: 0.4973
Epoch [70/100], Loss: 0.4743
Epoch [80/100], Loss: 0.4555
Epoch [90/100], Loss: 0.4411
Epoch [100/100], Loss: 0.4299
mse: 0.2233
